In [1]:
import pandas as pd

from data_preparation import load_match_summary
from data_preparation import load_player_stats
from data_preparation import aggregate_player_to_match_stats
from data_preparation import load_team_info
from data_preparation import load_venue_info
from data_preparation import create_match_summary_stats

from player import Player
from match import Match
from team import Team
from rating_calculator import PlayerRatingCalculator, TeamRatingCalculator
from projector import Projector

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.precision', 4)

1. load expected vaep data and player data
2. simulate initial updating process
3. put into a function
4. create objective function
5. run optuna optimisation to find update parameters

Load in player and match stats

In [2]:
match_summary = load_match_summary()
player_stats = load_player_stats()
match_stats = aggregate_player_to_match_stats(player_stats)
team_info, home_team_info, away_team_info = load_team_info()
venue_info, away_venue_info = load_venue_info()
match_summary_stats = create_match_summary_stats(match_summary, match_stats, home_team_info, away_team_info, away_venue_info)

In [6]:
match_summary.tail(1)

,Home_Team,Away_Team,Q4_Score,Margin,Total Game Score,Home Win,Venue,City,Date,Attendance,Temperature,Weather_Type,Year,Round_ID,Match_ID,Season,Home_Score,Home_Goals,Home_Behinds,Home_Shots,Home_Conversion,Away_Score,Away_Goals,Away_Behinds,Away_Shots,Away_Conversion
623,Collingwood,Greater Western Sydney,8.10.58 - 8.9.57,1,115,1.0,M.C.G.,Melbourne,2023-09-22 19:50:00,0,16,MOSTLY_SUNNY,2023,2023F3,2023F3_Collingwood_GreaterWesternSydney,2023,58,8,10,18,0.4444,57,8,9,17,0.4706


In [4]:
player_stats.tail(1)

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
28954,2023F4_Collingwood_BrisbaneLions,Collingwood,Will Hoskin-Elliott,2023F4,Will_Hoskin-Elliott,MIDFIELDER_FORWARD,CD_I291720,30,186,82,32,RIGHT,VIC,2011.0,2012.0,North Sunshine (Vic)/Western U18/GWS,4.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1993-09-02,89,0,0.0,0.0,0,2,0.0,0.0,0.0,0,0.0,7,0.0,0.0,0.0,0.0,50.0,12,58,0.0,0.0,0.0,0,0,0.0,1,0,0.0,1,0,0.0,0.0,0.0,3,0.0,5,0.0,0.0,0.0,11,0.0,5,0,0.0,175.0,4,0.0,0.0,1,0.0,5,0.0,0,0.0,0,2,0,0,13,1,6,0,0.0,Interchange,FINAL_TEAM,Interchange,Interchange,0.0,0.0,2023,0.0,0.0556,0.0207,0.1342,0.3983,-0.264,2.1038,3.4454,-1.3417,0.1191,0.141,-0.0627,1.4942,Will Hoskin-Elliott_2023,0,0.0,Collingwood,Brisbane Lions,Brisbane Lions,27,27,202327


In [5]:
match_stats.tail(1)

,Away_xScore,Home_xScore,Away_vaep_value,Home_vaep_value,Away_offensive_value,Home_offensive_value,Away_defensive_value,Home_defensive_value,Away_exp_vaep_value,Home_exp_vaep_value,Away_exp_offensive_value,Home_exp_offensive_value,Away_exp_defensive_value,Home_exp_defensive_value
Match_ID,,,,,,,,,,,,,,
2023F4_Collingwood_BrisbaneLions,69.6384,87.5739,15.5517,14.7012,13.1094,14.8605,2.4423,-0.1592,75.3591,94.1547,66.7313,101.0711,8.6278,-6.9164


Updating Process:

1. Sum up player values from each game as the actual offensive rating for that team
2. Get difference in match expected vaep values as the actual "Team Rating" for that match
3. Take into account home advantage for home team
4. Update long term player ratings
5. Update long term team ratings
5. Calculate "Defensive Rating" as the "Offensive Rating" + "Team Rating" (just for completeness)

Calculate Home Advantage (Rough Overall Average)

In [7]:
overall_ha = (match_summary_stats['Home_exp_vaep_value'].mean() - match_summary_stats['Away_exp_vaep_value'].mean()).round(2)
overall_ha

5.83

## Initialise Geelong

- Set all players ratings to 0
- Offensive Rating is sum of top 22 players
- Set Team Rating to 0
- Defensive Rating is difference between Offensive Rating and Team Rating

In [ ]:
geelong_players = list(player_stats[player_stats['Team'] == "Geelong"]['Player'].unique())
geelong_players_ratings = pd.DataFrame(geelong_players, columns =['Player'])
geelong_players_ratings['Rating'] = 0
geelong_offensive_rating = geelong_players_ratings.sort_values(by = "Rating").head(22)['Rating'].sum()
geelong_team_rating = 0
geelong_defensive_rating = geelong_offensive_rating - geelong_team_rating

In [ ]:
brisbane_players = list(player_stats[player_stats['Team'] == "Brisbane Lions"]['Player'].unique())
brisbane_players_ratings = pd.DataFrame(brisbane_players, columns =['Player'])
brisbane_players_ratings['Rating'] = 0
brisbane_offensive_rating = brisbane_players_ratings.sort_values(by = "Rating").head(22)['Rating'].sum()
brisbane_team_rating = 0
brisbane_defensive_rating = brisbane_offensive_rating - brisbane_team_rating

## First Round

Get player stats for match

In [ ]:
round_id = '202102'
round_player_stats = player_stats[player_stats['Round_ID'] == round_id]
round_match_id_list = list(round_player_stats['Match_ID'].unique())
match_id = [x for x in round_match_id_list if "Geelong" in x][0]
match_player_stats = round_player_stats[round_player_stats['Match_ID'] == match_id]

Get Geelong's player values for the match. Sum of these will be the team's Offensive Rating for the match

In [ ]:
geelong_player_stats = match_player_stats[match_player_stats['Team'] == "Geelong"]
geelong_match_offensive_rating = (geelong_player_stats['exp_vaep_value'].sum()).round(3)
geelong_match_offensive_rating

Get Opponent's player values for the match. Difference between the team's will be Geelong's Team Rating for the match.

In [ ]:
opp_player_stats = match_player_stats[match_player_stats['Team'] != "Geelong"]
geelong_match_team_rating = (geelong_match_offensive_rating - opp_player_stats['exp_vaep_value'].sum()).round(3)
geelong_match_team_rating

Calculate Geelong's Defensive Rating from Offensive and Team Ratings.

In [ ]:
geelong_match_defensive_rating = geelong_match_offensive_rating-geelong_match_team_rating

Update Player Ratings with Player Match Ratings

- Need to optimise the standard deviations for the prior ratings v the match ratings.
- Set to 10 / 25 for example.
- Want the standard deviations to reduce over time across the season
- Want the standard deviations to reset between seasons (increase uncertainty).
- Want the Ratings to revert to mean/regress between seasons.

Create Player Objects

In [ ]:
from player import Player

In [ ]:
PatrickDangerfield = Player('Patrick Dangerfield', 'Geelong')

Create Team Objects

In [ ]:
geelong_players = list(player_stats[player_stats['Team'] == "Geelong"]['Player'].unique())
geelong_players_list = [Player(x, "Geelong") for x in geelong_players]
geelong_player_dict = { k:v for (k,v) in zip(geelong_players, geelong_players_list)}
Geelong = Team("Geelong", geelong_player_dict)

In [ ]:
brisbane_players = list(player_stats[player_stats['Team'] == "Brisbane Lions"]['Player'].unique())
brisbane_players_list = [Player(x, "Brisbane Lions") for x in brisbane_players]
brisbane_player_dict = { k:v for (k,v) in zip(brisbane_players, brisbane_players_list)}
BrisbaneLions = Team("Brisbane Lions", brisbane_player_dict)

Create Match objects

In [ ]:
from typing import List

In [ ]:
round_id = '202102'
round_player_stats = player_stats[player_stats['Round_ID'] == round_id]
round_match_id_list = list(round_player_stats['Match_ID'].unique())
match_id = [x for x in round_match_id_list if "Geelong" in x][0]
match_player_stats = round_player_stats[round_player_stats['Match_ID'] == match_id]

In [ ]:
GeelongBrisbaneLions202102 = Match("202102_Geelong_BrisbaneLions", match_player_stats, player_stats, Geelong, BrisbaneLions)

Create Rating Calculator

Testing Team Rating Updates

In [ ]:
Geelong = Team("Geelong", geelong_player_dict)
Geelong.add_rating('202101', 0)
BrisbaneLions = Team("Brisbane Lions", brisbane_player_dict)
BrisbaneLions.add_rating('202101', 0)
GeelongBrisbaneLions202102 = Match("202102_Geelong_BrisbaneLions", match_player_stats, player_stats, Geelong, BrisbaneLions)

In [ ]:
Geelong.rating, BrisbaneLions.rating

In [ ]:
geelong_rating_calculator = TeamRatingCalculator(Geelong)
geelong_rating_calculator.update_team_rating(GeelongBrisbaneLions202102)

In [ ]:
brisbane_rating_calculator = TeamRatingCalculator(BrisbaneLions)
brisbane_rating_calculator.update_team_rating(GeelongBrisbaneLions202102)

In [ ]:
Geelong.rating, BrisbaneLions.rating

Player Updates

In [ ]:
PatrickDangerfield = Player('Patrick Dangerfield', Geelong)
PatrickDangerfieldCalculator = PlayerRatingCalculator(PatrickDangerfield)
PatrickDangerfield.add_rating('202101', 0)
# PatrickDangerfieldCalculator.get_player_rating('202101')
# PatrickDangerfieldCalculator.get_match_rating(GeelongBrisbaneLions202102)
# PatrickDangerfieldCalculator.update_player_rating(GeelongBrisbaneLions202102)
# PatrickDangerfield.rating

In [ ]:
TomHawkins = Player('Tom Hawkins', Geelong)
TomHawkinsCalculator = PlayerRatingCalculator(TomHawkins)
TomHawkins.add_rating('202101', 0)
# TomHawkinsCalculator.get_player_rating('202101')
# TomHawkinsCalculator.get_match_rating(GeelongBrisbaneLions202102)
TomHawkinsCalculator.update_player_rating(GeelongBrisbaneLions202102, prior_std=20, actual_std=30)
TomHawkins.rating

In [ ]:
Geelong.offensive_rating

Match Projector

Second Round

Third Round

End of Season